In [3675]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [3676]:
diabetes_df = pd.read_csv("diabetic_data.csv")

In [3677]:
# from sklearn.model_selection import train_test_split

# class ReadmissionPredictionDataset(torch.utils.data.Dataset):
#     def __init__(self, diabetes_df: pd.DataFrame, verbose = False):
#         diabetes_df['readmitted'] = diabetes_df['readmitted'].replace({'<30': 'YES', '>30': 'YES'})
        
        
#         diabetes_df = diabetes_df.drop_duplicates(subset='patient_nbr') # try keep = 'last'
#         diabetes_df = diabetes_df.drop(columns = 'patient_nbr')
#         diabetes_df = diabetes_df.drop(columns = 'medical_specialty')
#         diabetes_df = diabetes_df.drop(columns = 'weight')
#         diabetes_df = diabetes_df.drop(columns = 'payer_code')

#         # i originally kept these but they recommend dropping them:
#         diabetes_df = diabetes_df.drop(columns = 'admission_type_id')
#         diabetes_df = diabetes_df.drop(columns = 'discharge_disposition_id')
#         diabetes_df = diabetes_df.drop(columns = 'admission_source_id')

#         # diabetes_df = diabetes_df.drop(columns = 'examide') # all values the same
#         # diabetes_df = diabetes_df.drop(columns = 'citoglipton') # all values the a same
#         # diabetes_df = diabetes_df.drop(columns = 'acetohexamide')
#         # diabetes_df = diabetes_df.drop(columns = 'glimepiride-pioglitazon')
#         # diabetes_df = diabetes_df.drop(columns = 'metformin-rosiglitazone')

#         medications_to_drop = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
#                    'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone',
#                    'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin',
#                    'glyburide-metformin', 'glipizide-metformin', 'metformin-pioglitazone', 
#                    'metformin-rosiglitazone', 'glimepiride-pioglitazon', 'acetohexamide', 'citoglipton', 'examide']
#         diabetes_df = diabetes_df.drop(columns=medications_to_drop)

#         diabetes_df = diabetes_df[diabetes_df['gender'] != 'Unknown/Invalid']
#         diabetes_df = diabetes_df[diabetes_df['race'] != '?']
#         diabetes_df = diabetes_df[diabetes_df['diag_1'] != '?']

#         diabetes_df = diabetes_df.drop(columns = 'diag_2')
#         diabetes_df = diabetes_df.drop(columns = 'diag_3')

#         diabetes_df.loc[diabetes_df['number_diagnoses'] > 9, 'number_diagnoses'] = 9 # only a few values above 9
        
#         diabetes_df = diabetes_df.drop(columns = 'num_lab_procedures') #not sure if we drop this
#         diabetes_df = diabetes_df.drop(columns = 'number_emergency')

#         diabetes_df = pd.get_dummies(diabetes_df, columns=['gender'], dtype='int', prefix='is', prefix_sep='')

#         # remove the highly correlated data, they didn't give specific columns



#         diabetes_df = diabetes_df.drop(columns = 'encounter_id')
        
#         #diabetes_df = diabetes_df.drop(columns = 'diag_1')
        
        
#         # diabetes_df = diabetes_df.drop(columns = 'metformin')
#         # diabetes_df = diabetes_df.drop(columns = 'repaglinide')
#         # diabetes_df = diabetes_df.drop(columns = 'nateglinide')
#         # diabetes_df = diabetes_df.drop(columns = 'chlorpropamide')
#         # diabetes_df = diabetes_df.drop(columns = 'glimepiride')
#         # diabetes_df = diabetes_df.drop(columns = 'glipizide')
#         # diabetes_df = diabetes_df.drop(columns = 'glyburide')
#         # diabetes_df = diabetes_df.drop(columns = 'tolbutamide')
#         # diabetes_df = diabetes_df.drop(columns = 'pioglitazone')
#         # diabetes_df = diabetes_df.drop(columns = 'rosiglitazone')
#         # diabetes_df = diabetes_df.drop(columns = 'acarbose')
#         # diabetes_df = diabetes_df.drop(columns = 'miglitol')
#         # diabetes_df = diabetes_df.drop(columns = 'troglitazone')
#         # diabetes_df = diabetes_df.drop(columns = 'tolazamide')
        
#         # diabetes_df = diabetes_df.drop(columns = 'insulin')
#         # diabetes_df = diabetes_df.drop(columns = 'glyburide-metformin')
#         # diabetes_df = diabetes_df.drop(columns = 'glipizide-metformin')
#         # diabetes_df = diabetes_df.drop(columns = 'metformin-pioglitazone')

#         # diabetes_df = diabetes_df.drop(columns = 'number_diagnoses')
#         diabetes_df = diabetes_df.drop(columns = 'number_outpatient')

        
#         #diabetes_df = diabetes_df.drop(columns = 'num_procedures')
#         #diabetes_df = diabetes_df.drop(columns = 'num_medications')

#         # diabetes_df = diabetes_df[diabetes_df['diag_2'] != '?']
#         # diabetes_df = diabetes_df[diabetes_df['diag_3'] != '?']
        
#         #diabetes_df = diabetes_df[diabetes_df['weight'] != '?']
#         # diabetes_df = diabetes_df[diabetes_df['admission_type_id'] != 5]
#         # diabetes_df = diabetes_df[diabetes_df['admission_type_id'] != 6]
#         # diabetes_df = diabetes_df[diabetes_df['admission_source_id'] != 17]
#         # 11, 13, 14, 19, 20, 21
#         # diabetes_df = diabetes_df[diabetes_df['discharge_disposition_id'] != 11]
#         # diabetes_df = diabetes_df[diabetes_df['discharge_disposition_id'] != 13]
#         # diabetes_df = diabetes_df[diabetes_df['discharge_disposition_id'] != 14]
#         # diabetes_df = diabetes_df[diabetes_df['discharge_disposition_id'] != 19]
#         # diabetes_df = diabetes_df[diabetes_df['discharge_disposition_id'] != 20]
#         # diabetes_df = diabetes_df[diabetes_df['discharge_disposition_id'] != 21]

#         diabetes_df = diabetes_df[~diabetes_df['diag_1'].str.contains('V')]
#         diabetes_df = diabetes_df[~diabetes_df['diag_1'].str.contains('E')]
#         # diabetes_df = diabetes_df[~diabetes_df['diag_2'].str.contains('V')]
#         # diabetes_df = diabetes_df[~diabetes_df['diag_3'].str.contains('V')]

#         diabetes_df = pd.get_dummies(diabetes_df, columns=['gender'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['race'], dtype='int', prefix='is', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['admission_type_id'], dtype='int', prefix='is', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['admission_source_id'], dtype='int', prefix='is', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['discharge_disposition_id'], dtype='int', prefix='is', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['diag_1'], prefix='is', dtype='int', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['diag_2'], prefix='is', dtype='int', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['diag_3'], prefix='is', dtype='int', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['max_glu_serum'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['A1Cresult'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['metformin'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['repaglinide'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['nateglinide'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['chlorpropamide'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['glimepiride'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['glipizide'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['glyburide'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['tolbutamide'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['pioglitazone'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['rosiglitazone'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['acarbose'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['miglitol'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['troglitazone'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['tolazamide'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['insulin'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['glyburide-metformin'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['glipizide-metformin'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['metformin-pioglitazone'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['change'], dtype='int', prefix='is', prefix_sep='')
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['diabetesMed'], dtype='int', prefix='is', prefix_sep='')

#         age_mapping = {
#             '[0-10)': 5,
#             '[10-20)': 15,
#             '[20-30)': 25,
#             '[30-40)': 35,
#             '[40-50)': 45,
#             '[50-60)': 55,
#             '[60-70)': 65,
#             '[70-80)': 75,
#             '[80-90)': 85,
#             '[90-100)': 95
#         }
#         diabetes_df['age'] = diabetes_df['age'].replace(age_mapping)
#         # weight_mapping = {
#         #     '[0-25)': 12.5,
#         #     '[25-50)': 37.5,
#         #     '[50-75)': 62.5,
#         #     '[75-100)': 87.5,
#         #     '[100-125)': 112.5,
#         #     '[125-150)': 137.5,
#         #     '[150-175)': 162.5,
#         #     '[175-200)': 187.5
#         # }
#         # diabetes_df['weight'] = diabetes_df['weight'].replace(weight_mapping)

#         diag_mapping = {
#             'Infectious': [str(i) for i in range(1, 140)],
#             'Neoplasms': [str(i) for i in range(140, 240)],
#             'Endocrine': [str(i) for i in range(240, 280)],
#             'Circulatory': [str(i) for i in range(390, 460)],
#             'Respiratory': [str(i) for i in range(460, 520)],
#             'Digestive': [str(i) for i in range(520, 580)],
#             'Musculoskeletal': [str(i) for i in range(710, 740)],
#             'Genitourinary': [str(i) for i in range(580, 630)],
#             'Nervous': [str(i) for i in range(320, 390)],
#             'Symptoms': [str(i) for i in range(780, 800)]
#         }

#         def map_to_group(code):
#             for group, code_range in diag_mapping.items():
#                 if code in code_range:
#                     return group
#             return 'Other'

#         diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(map_to_group)
#         # diabetes_df['diag_2'] = diabetes_df['diag_2'].apply(map_to_group)
#         # diabetes_df['diag_3'] = diabetes_df['diag_3'].apply(map_to_group)
        
#         diabetes_df = pd.get_dummies(diabetes_df, columns=['diag_1'], dtype='int', prefix='is', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['diag_2'], dtype='int', prefix='is', prefix_sep='')
#         # diabetes_df = pd.get_dummies(diabetes_df, columns=['diag_3'], dtype='int', prefix='is', prefix_sep='')


#         readmit_mapping = {
#             'YES': 1,
#             'NO': 0
#         }
#         diabetes_df['readmitted'] = diabetes_df['readmitted'].replace(readmit_mapping)

#         # groups = diabetes_df.groupby('readmitted')
#         # counts = groups.count()
#         # print(counts)

#         # unique_values = diabetes_df['diag_1'].unique()
#         # print(unique_values)
#         # groups_type = diabetes_df.groupby('age')
#         # counts_type = groups_type.count()
#         # print(counts_type)
#         #counts['encounter_id'].plot.bar()

#         #data_x =  diabetes_df.loc[:, diabetes_df.columns[:len(diabetes_df.columns) - 1]]

#         data_x = diabetes_df.select_dtypes(include=[int, float]).drop('readmitted', axis=1)

#         data_y = diabetes_df['readmitted']

#         self.input = torch.tensor(data_x.values).type(torch.float32)
#         self.df = diabetes_df

#         self.output = torch.tensor(data_y.values).type(torch.float32)
    
#     def __len__(self):
#         return len(self.input)

#     def __getitem__(self, idx):
#         return (self.input[idx], self.output[idx])


# X = ReadmissionPredictionDataset(diabetes_df)
# # train_dataset, test_dataset = train_test_split(X, test_size=1)


# train_dataset, test_dataset = train_test_split(X, test_size=0.5, random_state=42)

# # train_dataset = ReadmissionPredictionDataset(train_df, verbose = True)
# # test_dataset = ReadmissionPredictionDataset(test_df)

# # train_df.head()
# #print(diabetes_df.columns)


# # input_size = len(train_dataset.input[0])
# # print(input_size)
# # print(len(test_dataset.input[0]))

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1000, shuffle = True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1000, shuffle = True)


In [3678]:
from sklearn.model_selection import train_test_split

class ReadmissionPredictionDataset(torch.utils.data.Dataset):
    def __init__(self, diabetes_df: pd.DataFrame, verbose = False):
        diabetes_df['readmitted'] = diabetes_df['readmitted'].replace({'<30': 'Yes', '>30': 'No'})
        
        diabetes_df = diabetes_df.drop_duplicates(subset='patient_nbr', keep='last') # try keep = 'last'
        
        columns_to_drop = ['patient_nbr', 'medical_specialty', 'weight', 'payer_code', 'encounter_id', 'diag_2', 'diag_3', 'number_emergency', 'number_inpatient', 'number_outpatient']
        diabetes_df = diabetes_df.drop(columns=columns_to_drop)

        medications_to_drop = ['repaglinide', 'nateglinide', 'chlorpropamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
                   'glyburide-metformin', 'glipizide-metformin', 'metformin-pioglitazone', 
                   'metformin-rosiglitazone', 'glimepiride-pioglitazone', 'acetohexamide', 'citoglipton', 'examide', 'tolbutamide', 'glimepiride', 'glipizide', 
                    'glyburide', 'pioglitazone', 'rosiglitazone']
        diabetes_df = diabetes_df.drop(columns=medications_to_drop)

        for column in diabetes_df:
            diabetes_df = diabetes_df[diabetes_df[column] != 'Unknown/Invalid']
            diabetes_df = diabetes_df[diabetes_df[column] != '?']

        values_to_remove = [11, 13, 14, 19, 20, 21] # remove people that died
        diabetes_df = diabetes_df[~diabetes_df['discharge_disposition_id'].isin(values_to_remove)]

        admission_mapping = {
            2: 'Emergency',
            1: 'Emergency',
            7: 'Emergency',
            6: 'Other',
            5: 'Other',
            8: 'Other',
            3: 'Elective',
            4: 'Newborn'
        }
        diabetes_df['admission_type_id'] = diabetes_df['admission_type_id'].replace(admission_mapping)

        diabetes_df['admission_source_id'] = diabetes_df['admission_source_id'].map({
            1: 'Physician Referral',
            2: 'Physician Referral',
            3: 'Physician Referral',
            4: 'Other',
            5: 'Other',
            6: 'Other',
            7: 'Emergency Room',
            8: 'Other',
            9: 'Other',
            10: 'Other',
            11: 'Other',
            12: 'Other',
            13: 'Other',
            14: 'Other',
            15: 'Other',
            17: 'Other',
            18: 'Other',
            19: 'Other',
            20: 'Other',
            21: 'Other',
            22: 'Other',
            23: 'Other',
            24: 'Other',
            25: 'Other',
            26: 'Other'
        })
        diabetes_df['admission_source_id'] = diabetes_df['admission_source_id'].replace(admission_mapping)
    
        diabetes_df['discharge_disposition_id'] = diabetes_df['discharge_disposition_id'].apply(lambda x: 'Home' if x == 1 else 'Other')

        diag_mapping = {
            'Infectious': [str(i) for i in range(1, 140)],
            'Neoplasms': [str(i) for i in range(140, 240)],
            'Endocrine': [str(i) for i in range(240, 280)],
            'Circulatory': [str(i) for i in range(390, 460)],
            'Respiratory': [str(i) for i in range(460, 520)],
            'Digestive': [str(i) for i in range(520, 580)],
            'Musculoskeletal': [str(i) for i in range(710, 740)],
            'Genitourinary': [str(i) for i in range(580, 630)],
            'Nervous': [str(i) for i in range(320, 390)],
            'Symptoms': [str(i) for i in range(780, 800)]
        }
        def map_to_group(code):
            for group, code_range in diag_mapping.items():
                if code in code_range:
                    return group
            return 'Other'
        diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(map_to_group)

        

        diabetes_df.dropna(subset=['A1Cresult'], inplace=True)
        diabetes_df.dropna(subset=['max_glu_serum'], inplace=True)

        columns_to_convert = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'metformin', 'insulin', 'diag_1', 'gender', 'race', 'max_glu_serum', 'A1Cresult']
        for column in columns_to_convert:
            diabetes_df = pd.get_dummies(diabetes_df, columns=[column], dtype='int')

        age_mapping = {
            '[0-10)': 5,
            '[10-20)': 15,
            '[20-30)': 25,
            '[30-40)': 35,
            '[40-50)': 45,
            '[50-60)': 55,
            '[60-70)': 65,
            '[70-80)': 75,
            '[80-90)': 85,
            '[90-100)': 95
        }
        diabetes_df['age'] = diabetes_df['age'].replace(age_mapping)

        readmit_mapping = {
            'Yes': 1,
            'no': 0,
            'No': 0,
            'YES' : 1,
            'Ch': 1,
            'NO': 0
        }
        diabetes_df['readmitted'] = diabetes_df['readmitted'].replace(readmit_mapping)
        diabetes_df['diabetesMed'] = diabetes_df['diabetesMed'].replace(readmit_mapping)
        diabetes_df['change'] = diabetes_df['change'].replace(readmit_mapping)

        def outliers_remover(df, columns_to_process):
            aa = []
            for column_name in columns_to_process:
                if pd.api.types.is_numeric_dtype(df[column_name]):
                    column = df[column_name]
                    q1 = column.quantile(0.25)
                    q3 = column.quantile(0.75)
                    iqr = q3 - q1
                    upper = q3 + 1.5 * iqr
                    lower = q1 - 1.5 * iqr
                    outliers = (column > upper) | (column < lower)
                    aa.extend(outliers[outliers].index)  # Append the index of outliers
            df = df.drop(aa).reset_index(drop=True)
            return df
        
        columns_to_process = ['num_medications', 'num_procedures', 'num_lab_procedures', 'time_in_hospital']

# Apply the outliers_remover function to the specified columns in diabetes_df
        diabetes_df = outliers_remover(diabetes_df, columns_to_process)

        # print(diabetes_df.head())
        # unique_values = diabetes_df['max_glu_serum'].unique()
        # print(unique_values)
        groups = diabetes_df.groupby('readmitted')
        counts = groups.count()
        print(counts)

        minority_class = diabetes_df[diabetes_df['readmitted'] == 1]
        majority_class = diabetes_df[diabetes_df['readmitted'] == 0]
        print(len(majority_class))
        print(len(minority_class))
        # Upsample the minority class to match the size of the majority class
        if len(minority_class) < len(majority_class):
            minority_class = resample(minority_class, replace=True, n_samples=len(majority_class))
            
        diabetes_df = pd.concat([majority_class, minority_class])

        
        data_x = diabetes_df.select_dtypes(include=[int, float]).drop('readmitted', axis=1)

        data_y = diabetes_df['readmitted']



        #print(diabetes_df)

        self.input = torch.tensor(data_x.values).type(torch.float32)
        self.df = diabetes_df

        self.output = torch.tensor(data_y.values).type(torch.float32)
    
    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        return (self.input[idx], self.output[idx])


X = ReadmissionPredictionDataset(diabetes_df)
#print(X.df)
train_dataset, test_dataset = train_test_split(X, test_size=.2, random_state=42)

# train_df, test_df = train_test_split(diabetes_df, test_size=0.2, random_state=42)

# train_dataset = ReadmissionPredictionDataset(train_df, verbose = True)
# test_dataset = ReadmissionPredictionDataset(test_df)



train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 300, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 300, shuffle = True)


            age  time_in_hospital  num_lab_procedures  num_procedures  \
readmitted                                                              
0           175               175                 175             175   
1            16                16                  16              16   

            num_medications  number_diagnoses  change  diabetesMed  \
readmitted                                                           
0                       175               175     175          175   
1                        16                16      16           16   

            admission_type_id_Elective  admission_type_id_Emergency  \
readmitted                                                            
0                                  175                          175   
1                                   16                           16   

            admission_type_id_Other  discharge_disposition_id_Home  \
readmitted                                                           
0

In [3679]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layers = nn.Sequential(nn.Linear(47,16), nn.Sigmoid(), nn.Linear(16,32), nn.Sigmoid(), nn.Linear(32, 64), nn.Sigmoid(), nn.Linear(64, 1), nn.Sigmoid())
    # def __init__(self):
    #     super(SimpleNet, self).__init__()
    #     self.layers = nn.Sequential(nn.Conv2d(1, 32, kernel_size=(1, 3)), nn.ReLU(), nn.MaxPool2d(kernel_size=(1, 2)), nn.Conv2d(32, 64, kernel_size=(1, 3)), nn.ReLU(), nn.MaxPool2d(kernel_size=(1, 2)), nn.Conv2d(64, 128, kernel_size=(1, 3)), nn.ReLU(), nn.MaxPool2d(kernel_size=(1, 2)), nn.Flatten(), nn.Linear(896, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU(), nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 1), nn.Sigmoid())
    
    
    def forward(self, x):
        yhat = self.layers(x)
        return yhat

In [3680]:
from tqdm.notebook import tqdm

def train_network(model, train_loader, criterion, optimizer, nepoch=100):
    try:
        for epoch in tqdm(range(nepoch)):
            print('EPOCH %d'%epoch)
            total_loss = 0
            count = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                count += 1
            print('{:>12s} {:>7.5f}'.format('Train loss:', total_loss/count))
    except KeyboardInterrupt:
        print('Exiting from training early')
    return

In [3681]:
def test_network(model, test_loader):
    correct = 0
    total = 0
    true, pred = [], []
    with torch.no_grad():
        for inputs, labels  in test_loader:
            outputs = model(inputs)
            predicted = torch.round(outputs).squeeze()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            true.append(labels)
            pred.append(predicted)
    acc = (100 * correct / total)
    print('accuracy: %0.3f' % (acc))
    true = np.concatenate(true)
    pred = np.concatenate(pred)
    return acc, true, pred

In [3682]:
model = SimpleNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.01)

In [3683]:
train_network(model, train_loader, criterion, optimizer, nepoch=400)  

  0%|          | 0/400 [00:00<?, ?it/s]

EPOCH 0
 Train loss: 0.71376
EPOCH 1
 Train loss: 0.70339
EPOCH 2
 Train loss: 0.69758
EPOCH 3
 Train loss: 0.69483
EPOCH 4
 Train loss: 0.69366
EPOCH 5
 Train loss: 0.69322
EPOCH 6
 Train loss: 0.69305
EPOCH 7
 Train loss: 0.69300
EPOCH 8
 Train loss: 0.69299
EPOCH 9
 Train loss: 0.69299
EPOCH 10
 Train loss: 0.69299
EPOCH 11
 Train loss: 0.69300
EPOCH 12
 Train loss: 0.69301
EPOCH 13
 Train loss: 0.69301
EPOCH 14
 Train loss: 0.69302
EPOCH 15
 Train loss: 0.69303
EPOCH 16
 Train loss: 0.69304
EPOCH 17
 Train loss: 0.69304
EPOCH 18
 Train loss: 0.69305
EPOCH 19
 Train loss: 0.69305
EPOCH 20
 Train loss: 0.69305
EPOCH 21
 Train loss: 0.69305
EPOCH 22
 Train loss: 0.69305
EPOCH 23
 Train loss: 0.69305
EPOCH 24
 Train loss: 0.69305
EPOCH 25
 Train loss: 0.69305
EPOCH 26
 Train loss: 0.69304
EPOCH 27
 Train loss: 0.69304
EPOCH 28
 Train loss: 0.69304
EPOCH 29
 Train loss: 0.69304
EPOCH 30
 Train loss: 0.69303
EPOCH 31
 Train loss: 0.69303
EPOCH 32
 Train loss: 0.69302
EPOCH 33
 Train loss

In [3687]:
acc, true, pred = test_network(model, test_loader)
print(pred)
# print(train_dataset)

accuracy: 95.714
[1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1.]


In [3685]:
print(X.df.dtypes)

age                                       int64
time_in_hospital                          int64
num_lab_procedures                        int64
num_procedures                            int64
num_medications                           int64
number_diagnoses                          int64
change                                    int64
diabetesMed                               int64
readmitted                                int64
admission_type_id_Elective                int32
admission_type_id_Emergency               int32
admission_type_id_Other                   int32
discharge_disposition_id_Home             int32
discharge_disposition_id_Other            int32
admission_source_id_Emergency Room        int32
admission_source_id_Physician Referral    int32
metformin_Down                            int32
metformin_No                              int32
metformin_Steady                          int32
metformin_Up                              int32
insulin_Down                            